# Unsupervised learning

L'objectif de ce TP est de tester et de comprendre les différentes méthodes de ML non supervisé. Dans ce TP, vous devez remplacer les ... par le code adéquat. La seed aléatoire est fixée pour les différent TP, après les avoir complétés n'hésitez pas à la changer.

- [Dimensionnality reduction](#Dimensionality-reduction)
  - [PCA](#PCA)
  - [Manifold Learning](#Manifold-Learning)
- [Clustering](#Clustering)
  - [KMeans](#KMeans)
  - [DBSCAN](#DBSCAN)
  - [GMM](#GMM)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Réduction de dimension

### Création d'un jeu de donné

Pour commencer nous allons créer un jeu de donné constitué de points 2D corrélés

In [ ]:
# Creation d'un jeu de donnees avec deux axes corrélés
np.random.seed(42)
X = (np.random.rand(2, 2) @ np.random.randn(2, 200)).T
plt.scatter(X[:, 0], X[:, 1]);

### PCA
**Principal Component Analysis**

Pour commencer nous allons utiliser l'analyse en composante principale pour décomposer ces données. Plus d'informations sur la PCA de sklearn sont disponibles sur leur site internet : https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
from sklearn.decomposition import PCA


In [ ]:
# Créer un PCA avec 2 composantes et l'utiliser pour fitter X
# Vous pouvez ensuite accéder aux composantes principales avec pca.components_ et à la variance avec pca.explained_variance_
pca = ...
pca.fit(...)

In [ ]:
print("Composantes principales : ")
print(pca.components_)

print("Variance : ")
print(pca.explained_variance_)

#### Visualisation des composantes

En utilisant la fonction ci-dessous on peut visualiser les axes extraits par la PCA

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax if ax is not None else plt.gca()
    arrowprops = dict(arrowstyle='->', color='k', lw=2, shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

In [ ]:
# A l'aide la la fonction draw_vector, ont peux les composantes principales sur le plot précédent
plt.scatter(X[:, 0], X[:, 1])
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)

#### Transformation des données

On peut maintenant utiliser la PCA pour transformer les données selon les deux axes obtenus grâce à la méthode `fit_transform`. Les données résultantes doivent maintenant être décorrélées. 

In [ ]:
#Utiliser le PCA pour transformer X selon les composantes principales
...
X_trans = ...
plt.scatter(X_trans[:, 0], X_trans[:, 1])

#### Réduction de dimension

On peut aussi utiliser la PCA pour réduire la dimension de nos données. Pour cela effectuer une PCA avec moins de dimensions que nos données d'entrée.

In [ ]:
# En utilisant le PCA avec une dimension inférieur à celle de X, on peut réduire la dimension des données d'entrée
...
pca = ...
X_trans = ...
#Utiliser ensuite la méthode inverse_transform permet pour revenir dans l'espace d'origine
...
X2 = ...

In [ ]:
plt.scatter(X[:, 0], X[:, 1], label='données initiales')
plt.scatter(X2[:, 0], X2[:, 1], label='données après PCA')
plt.legend(frameon=False, loc=4)

### Manifold Learning

Nous allons maintenant essayer de réduire la dimension d'images de chiffres manuscrits à l'aide du PCA et de manifold learning.
Les images sont accessibles grâce à digit.images (matrice 8x8), une version linéarisée de ces images peut être obtenue avec digits.data (vecteur de 64 éléments).  

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()

In [ ]:
fig, axes = plt.subplots(3, 5, figsize=(10, 6), subplot_kw={'xticks': [], 'yticks': []})
for i, ax in enumerate(axes.flat):
    ax.imshow(digits.images[i], cmap='binary_r')
plt.grid(False)


#### PCA

Pour commencer utiliser le PCA pour réduire la dimension des données à 2 composantes. On pourra ainsi visualiser la séparation des chiffres.

In [ ]:
projected_data = ...

plt.figure(figsize=(10,6));
plt.scatter(projected_data[:, 0], projected_data[:, 1], c=digits.target, edgecolor='none', alpha=0.5, cmap=plt.colormaps['cubehelix'])
plt.colorbar()


#### t-SNE

Comme nous l'avons vu en cours, le PCA ne fonctionne que pour les données séparables linéairement. Nous allons donc essayer le t-distributed stochastic neighbor embedding pour séparer les chiffres.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# Utiliser le TSNE pour réduire la dimension des données, la syntaxe est similaire à celle du PCA

projected_data = ...
plt.scatter(projected_data[:, 0], projected_data[:, 1], c=digits.target, edgecolor='none', alpha=0.5, cmap=plt.cm.get_cmap('cubehelix', 10))
plt.colorbar()

## Clustering

Nous allons maintenant tester les techniques de clustering. Notre objectif est maintenant de rassembler ensemble les données similaires sans information a priori.

- K-means, DBSCAN

    https://scikit-learn.org/stable/modules/clustering.html
    
- Gaussian mixture

    https://scikit-learn.org/stable/modules/mixture.html

#### Donnée

Comme donnée pour tester les différents algorithmes de clustering, nous allons utiliser la fonction `make_blobs` de sklearn qui génère des paquets donnés selon des distributions gaussiennes. La fonction plot blob qui prend en entrée les données, leur label et le nombre de clusters nous permettent de les visualiser.

In [ ]:
from sklearn.datasets import make_blobs
np.random.seed(44)
X, labels = make_blobs(n_samples=100, centers=10, cluster_std=0.3)

In [ ]:
def plot_blobs(X, y=None, n=3):
    fig, ax = plt.subplots(figsize=(10, 8))
    if y is None:
        ax.scatter(X[:, 0], X[:, 1])
        ax.set_title("Raw data", fontsize=14)
    else:
        im = ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.get_cmap('viridis', n))
        ax.set_title("Labeled data", fontsize=14)

In [ ]:
plot_blobs(X)

In [ ]:
plot_blobs(X, labels, 10)

### KMeans

Commençons par utiliser la méthode du kmean. Toutes les méthodes de clustering de sklearn s'implémentent de la même manière. On définit les paramètres de notre modèle (ici avec `KMeans(...)`), on l'utilise pour fiter nos données avec `.fit(...)`, puis on accède au label de nos points à l'aide de `.predict(...)`.
Lors de la création du kmean 3 paramètres sont importants : `n_clusters` le nombre de clusters, `init` le type de méthode d'initialisation utilisé pour choisir nos points de départ et `n_init` le nombre d'initialisations différentes qui vont être testées.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# Utilisez KMeans pour prédire les labels des données X

kmeans = KMeans(...).fit(...)
y_kmeans = kmeans.predict(...)

In [ ]:
plot_blobs(...)

### DBSCAN

Utilisons maintenant DBScan avec nos données. L'interface est la même à la différence que l'on à pas besoin de préciser le nombre de clusters. Les deux paramètres importants lors de la création du DBScan sont : `eps` la distance entre deux points d'un même cluster et `min_samples` le nombre minimum de points nécessaire pour créer un cluster.

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
#Utiliser maintenant DBSCAN pour prédire les labels des données X 
dbscan = ...
y_dbscan = ...

print(f"n_clusters = {len(set(dbscan.labels_))}")

In [ ]:
# Puis plotter le résultat
plot_blobs(...)

## GMM

Pour finir, testons les GMM avec nos données de base. Il s'implémente exactement comme le kmean à la différence que le ne parle plus de cluster mais de composant `n_components`.

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
# Pour finir essayer la même chose avec un GaussianMixture

gmm = ...
y_gmm = ...

In [ ]:
plot_blobs(...)

### Application à des jeux de données plus complexe

Les données utilisées jusqu'ici étaient extrêmement simples, il est donc normal qu'on trouve les bons clusters.
Essayons maintenant avec trois jeux de données plus complexes. À vous de déterminer la solution la plus appropriée à chacun.

In [ ]:
# Blob avec des corrélations
np.random.seed(40)
Xi, labelsi = make_blobs(n_samples=300, centers=5, cluster_std=1.5)
X2 = Xi @ np.random.rand(2, 2)
plot_blobs(X2, labelsi, 5)

In [ ]:
...

In [ ]:
# Nombre de blob inconue
np.random.seed(111)
X3, labels3 = make_blobs(n_samples=200, centers=int(10+np.random.rand()*20), cluster_std=0.3)
plot_blobs(X3, labels3, labels3.max())

In [ ]:
# Blobs avec des variances différentes supposées
np.random.seed(44)
Xb, labelsb = make_blobs(n_samples=300, centers=3, cluster_std=2)
# Ajout de blobs sulémentaires
X4 = np.vstack([X, Xb])
labels4 = np.hstack([labels, labelsb+labels.max()+1])
plot_blobs(X4, labels4, 13)

In [ ]:
...